In [3]:
!nvidia-smi

Wed Jan 18 03:39:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.203.03   Driver Version: 450.203.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 라이브러리 및 파일 불러오기

In [4]:
# 우선 프로젝트 진행을 위한 라이브러리를 불러옵니다.

import os
import numpy as np
import pandas as pd
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/project/AIFFELTHON'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights/mcc_weights')
DATA_PATH = os.path.join('data/mcc')
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # connected to GPU if 'cuda' is printed here

cuda


In [ ]:
# checking imgs in a folder

for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    for i, filename in enumerate(filenames):
        print(os.path.join(dirpath, filename)) # prints file names
        image = Image.open(os.path.join(dirpath, filename), 'r')
        print(f'size: ({image.width}, {image.height}, {image.getbands()})') # prints img info
        plt.imshow(image)
        plt.show()
        if i==4:
            break # print 4 per folder

# Create Functions

In [7]:
# Normalize imgs, resize to 224x224
# Create pipeline
# PyTorch offers various augmentation techniques in torchvision.transforms.Compose

def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data

In [8]:
# creating train dataset

BATCH_SIZE = 64 # changed from 64 to 1

train_loader, _train_data = create_dataloader(TRAIN_PATH, BATCH_SIZE, True)
target_class_num = len(os.listdir(os.path.join(TRAIN_PATH)))

print('target_class_num: ', target_class_num)
print('train: ', _train_data.class_to_idx)

target_class_num:  5
train:  {'07_inner_cupholder_refined2_train_2000': 0, '08_inner_cupholder_dirt_refined2_train_892': 1, '11_inner_front_seat_train_2000': 2, '12_inner_rear_seat_train_2000': 3, '14_inner_sheet_dirt_train_2000': 4}


In [9]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TRAIN_PATH):
    print(f'{rootpath} : {len(filenames)}')

data/mcc/train : 0
data/mcc/train/07_inner_cupholder_refined2_train_2000 : 2000
data/mcc/train/14_inner_sheet_dirt_train_2000 : 2000
data/mcc/train/12_inner_rear_seat_train_2000 : 2000
data/mcc/train/11_inner_front_seat_train_2000 : 2000
data/mcc/train/08_inner_cupholder_dirt_refined2_train_892 : 892


In [10]:
# creating test dataset

BATCH_SIZE = 64 # changed from 64 to 1

test_loader, _test_data = create_dataloader(TEST_PATH, BATCH_SIZE, False)
target_class_num = len(os.listdir(os.path.join(TEST_PATH)))

print('target_class_num: ', target_class_num)
print('test: ', _test_data.class_to_idx)

target_class_num:  5
test:  {'07_inner_cupholder_refined2_test_500': 0, '08_inner_cupholder_dirt_refined2_test_200': 1, '11_inner_front_seat_test_500': 2, '12_inner_rear_seat_test_500': 3, '14_inner_sheet_dirt_test_500': 4}


In [11]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TEST_PATH):
    print(f'{rootpath} : {len(filenames)}')

data/mcc/test : 0
data/mcc/test/08_inner_cupholder_dirt_refined2_test_200 : 200
data/mcc/test/07_inner_cupholder_refined2_test_500 : 500
data/mcc/test/14_inner_sheet_dirt_test_500 : 500
data/mcc/test/12_inner_rear_seat_test_500 : 500
data/mcc/test/11_inner_front_seat_test_500 : 500


In [12]:
# metrics from sklearn.metrics

def calculate_metrics(trues, preds):
    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')
    return accuracy, f1, precision, recall

In [13]:
# train function

def train(dataloader, net, learning_rate, weight_decay_level, device):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        net.parameters(),
        lr = learning_rate, 
        weight_decay = weight_decay_level
    )

    net.train()

    train_losses = list()
    train_preds = list()
    train_trues = list()

    for idx, (img, label) in enumerate(dataloader):

        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()

        out = net(img)

        _, pred = torch.max(out, 1)
        loss = criterion(out, label)

        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_trues.extend(label.view(-1).cpu().numpy().tolist())
        train_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(train_trues, train_preds)

    print('\n''====== Training Metrics ======')
    print('Loss: ', mean(train_losses))
    print('Acc: ', acc)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', rec)
    print(confusion_matrix(train_trues, train_preds))

    return net, acc, f1, prec, rec

In [14]:
# test function

def test(dataloader, net, device):

    criterion = torch.nn.CrossEntropyLoss()
    
    net.eval()
    test_losses = list()
    test_trues = list()
    test_preds = list()
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(dataloader):

            img = img.to(device)
            label = label.to(device)

            out = net(img)

            _, pred = torch.max(out, 1)
            loss = criterion(out, label)

            test_losses.append(loss.item())
            test_trues.extend(label.view(-1).cpu().numpy().tolist())
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(test_trues, test_preds)

    print('====== Test Metrics ======')
    print('Test Loss: ', mean(test_losses))
    print('Test Acc: ', acc)
    print('Test F1: ', f1)
    print('Test Precision: ', prec)
    print('Test Recall: ', rec)
    print(confusion_matrix(test_trues, test_preds))

    return net, acc, f1, prec, rec

In [15]:
# code to save best params based on acc

def train_classifier(net, train_loader, test_loader, n_epochs, learning_rate, weight_decay, device):
    best_test_acc = 0
    
    model_save_path = None
    model_save_base = MODEL_PATH
    if not os.path.exists(model_save_base):
        os.makedirs(model_save_base)
    
    print('>> Start Training Model!')
    for epoch in range(n_epochs):
        
        print('> epoch: ', epoch)

        net, _, _, _, _ = train(train_loader, net, learning_rate, weight_decay, device)
        net, test_acc, _, _, _  = test(test_loader, net, device)

        if test_acc > best_test_acc:

            best_test_acc = test_acc
            test_acc_str = '%.5f' % test_acc

            print('[Notification] Best Model Updated!')
            model_save_path = os.path.join(model_save_base, 'mcc_classifier_acc_' + str(test_acc_str) + '.pth') 
            torch.save(net.state_dict(), model_save_path)
                
    return model_save_path

In [16]:
target_class_num

5

In [17]:
# load pre-trained resnet50

net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

net.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /aiffel/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
# result

EPOCHS = 20
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  1.1546497859543177
Acc:  0.5618533513270355
F1:  0.5001272530857539
Precision:  0.5499085699284951
Recall:  0.5100340807174888
[[1568   49  154  129  100]
 [ 539   84  111  103   55]
 [ 543   24  905  401  127]
 [ 269   18  284 1205  224]
 [ 305   24  143  294 1234]]
====== Test Metrics ======
Test Loss:  1.2229841305561024
Test Acc:  0.6372727272727273
Test F1:  0.556998833535972
Test Precision:  0.5982970781858645
Test Recall:  0.5692
[[448  13  37   1   1]
 [126  14  55   5   0]
 [125  27 331  17   0]
 [  0  58  58 383   1]
 [ 15  20  54 185 226]]
[Notification] Best Model Updated!
> epoch:  1

====== Training Metrics ======
Loss:  0.7292913693747074
Acc:  0.7415654520917678
F1:  0.6867978368166056
Precision:  0.7059311225321049
Recall:  0.6874726457399103
[[1729  100   76    8   87]
 [ 374  226   97   77  118]
 [ 133   39 1596  170   62]
 [  11   28  121 1599  241]
 [ 114   52   71  319 1444]]
====== Test M

====== Test Metrics ======
Test Loss:  0.2973345961437354
Test Acc:  0.9122727272727272
Test F1:  0.9200854780180183
Test Precision:  0.9378839537755823
Test Recall:  0.9192
[[350   1 131  18   0]
 [  0 194   6   0   0]
 [  0   0 490  10   0]
 [  0   0  16 484   0]
 [  0   0  11   0 489]]
> epoch:  14

====== Training Metrics ======
Loss:  0.06941968740948992
Acc:  0.9779577147998201
F1:  0.9795916468723057
Precision:  0.9796683201771798
Recall:  0.9795304932735427
[[1962    1   26    8    3]
 [   1  885    0    1    5]
 [  22    0 1922   52    4]
 [  11    0   32 1950    7]
 [   6    5    4    8 1977]]
====== Test Metrics ======
Test Loss:  0.41103519520856024
Test Acc:  0.8727272727272727
Test F1:  0.8836377171277199
Test Precision:  0.9001455336447899
Test Recall:  0.8879999999999999
[[492   2   3   0   3]
 [  0 200   0   0   0]
 [ 50   1 315 125   9]
 [ 86   0   0 414   0]
 [  0   0   1   0 499]]
> epoch:  15

====== Training Metrics ======
Loss:  0.06703815437022516
Acc:  0.980544